<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Statsmodels Lab

---

In [1]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=1.5)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

## Load the data

In [3]:
df = pd.read_csv('../../../../resource-datasets/auto_stats/Auto.csv')

## Do any necessary cleaning steps

## Use statsmodels to fit a linear regression model predicting `mpg`

### Use data frames

Describe what the model tells about 
- confidence intervals
- model quality
- residuals

### Use the formula language

Experiment with the formula language. Include interaction terms or nonlinear predictor dependence.

**Hint:** If you want to use patsy with categorical variables, you can dummify within the formula
with 

`formula = 'mpg~C(origin)'`

or

`formula = 'mpg~C(origin, Treatment(reference=2))'`

where the latter allows you to decide which category to use as the reference point rather than just dropping first.

### Fit with regularization

## Bonus: Construct an sklearn wrapper for statsmodels

Follow the example [here](http://nelsonauner.com/data/2018/05/21/wrap-statsmodels-in-sklearn.html) and use cross validation.

In [4]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score